In [1]:
import json
import re
import nltk
import emoji
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from wordcloud import WordCloud
from pysentimiento.preprocessing import preprocess_tweet

## Corpus SENT-COVID

In [2]:
with open('data/SENT-COVID.json') as file:
    data = json.load(file)
    
pd.options.mode.chained_assignment = None                                         
pd.set_option('display.max_colwidth',None)   


df = pd.DataFrame(data)
print('Numero de tweets: ' + str(len(df)))
df.head()

Numero de tweets: 4594


,Label,Tweet,id
0,NEUTRO,-@dulcema201 @BronstonRaqsa02 Protocolo de COVID !!!!,1401047081121353728
1,NEUTRO,-#COVID19 #QuedateEnCasa en Morelia Centro,1258159310162595843
2,POSITIVO,-México va en en aumento con el #Covid_19. Tal vez no tengamos la estabilidad de Europa o estados unidos. Para mantener 120 días en paro total. Pero podemos ser precavidos al usar la #SanaDistancia,1272748988626862082
3,NEUTRO,-@sororavirus Creo en todo y nada. 💜,1349385638722883585
4,NEGATIVO,-@GobiernoMX había prometido 389,1360615587114844161


## Preprocesamiento

In [3]:
def clean_tweet(text):
  text = re.sub(r'[~^0-9]', '', text) #numeros
  text = re.sub("\\s+", ' ', text) ##Espacios blancos dobles
  text = re.sub('\n', ' ', text) ##Saltos de linea

  pattern = r'([.])([A-Z#@¿])'
  pattern2 = r'([-])([a-zA-Z#@¿])'
  pattern3 = r'([a-zA-Z])([#@¿])'
  pattern4 = r'([:!])([a-zA-Z#@¿])'
  text = re.sub(pattern, r'\1 \2', text) # Separacion de punto seguido por una mayuscula
  text = re.sub(pattern2, r'\1 \2', text)
  text = re.sub(pattern3, r'\1 \2', text)
  text = re.sub(pattern4, r'\1 \2', text)
  return text 


def preprocess(text):
  return preprocess_tweet(text, shorten=2, emoji_wrapper='', user_token='')  # Preprocesamiento de pysentimiento   


def normalize(text):
 text = "".join(u for u in text if u not in ("?","¿", ".", ";", ":", "!","¡",'"',"%","“","”","$","&","'","\\", "(",")",
                                             "*","+",",","/","<",">","=","^","•","...", "ç","π","ⓘ", "-", "_","#","|"))
 a,b = 'áéíóúÁÉÍÓÚ','aeiouAEIOU'
 trans = str.maketrans(a,b)     
 text = text.translate(trans) # Reemplazo de palabras acentuadas       

 pattern = r'([a-z])([A-Z-])'
 text = re.sub(pattern, r'\1 \2', text)
 #text = re.sub(r'@[A-Za-z0-9_]+', '', text)
 text = text.lower()
 return text  


def tokenize(text):    
  text= text.split(sep = ' ')  # Tokenización por palabras individuales
  text= [token for token in text if len(token) > 1]  # Eliminación de tokens con una longitud < 2
  return(text) 

In [5]:
df['clean_tweet'] = df['Tweet'].apply(clean_tweet) 
df['preprocess_tweet'] = df['clean_tweet'].apply(preprocess)
df['norm_tweet'] = df['preprocess_tweet'].apply(normalize)
df['tokenized_tweet'] = df['norm_tweet'].apply(tokenize)

df[['Tweet','norm_tweet','Label']][95:105]

,Tweet,norm_tweet,Label
95,-#SNTEsalud ⚕️⚠️ ALERTA ⚠️ México vive alto contagio #coronavirus #COVID19 🦠 Ante síntomas no te automediques 💊🚫 ni tomes fármacos que prometen curar el #COVID 💊❌ y llama al 📲 800 00 44 800📱#QuedateEnCasa#FelizViernes #22DeMayo México #Sección35 #FelizFinde #RT https://t.co/wNIfhlJflO,sntesalud simbolo de medicina advertencia alerta advertencia mexico vive alto contagio coronavirus covid microbio ante sintomas no te automediques pildora prohibido ni tomes farmacos que prometen curar el covid pildora marca de cruz y llama al movil con una flecha telefono movil quedate en casa feliz viernes de mayo mexico seccion feliz finde rt url,NEGATIVO
96,-@LOVREGA @FelipeCalderon Pues como foca aplaudidora te queda bien el papel,pues como foca aplaudidora te queda bien el papel,NEGATIVO
97,"-""No habrá un """"regreso a la normalidad"""" en el mundo tras la pandemia de covid-19 https://t.co/bRO7munVtA vía @UniNoticias @dadams7308""",no habra un regreso a la normalidad en el mundo tras la pandemia de covid url via,NEGATIVO
98,-Ya le hizo daño la vacuna.Cuando yo vivía en Alemania todo era paz y tranquilidad,ya le hizo daño la vacuna cuando yo vivia en alemania todo era paz y tranquilidad,NEGATIVO
99,-Marcarle a mi preciosita en momento de crisis. 🥺🥺🥺,marcarle a mi preciosita en momento de crisis cara de por favor cara de por favor,POSITIVO
100,-@albertoviruete @nenulo Esos hermanos Negrete una bola de vividores,esos hermanos negrete una bola de vividores,NEGATIVO
101,"-""@rayados está en crisis- la Crisis https://t.co/aAYKlKFfFS""",esta en crisis la crisis url,NEUTRO
102,-Uff 😥 🦠🦠🦠#QuedateEnCasa🏡 #Coahuila #Mexico 😢,uff cara triste pero aliviada microbio microbio quedate en casa casa con jardin coahuila mexico cara llorando,NEGATIVO
103,-No sé qué tanto maman con #LadyVacuna,no se que tanto maman con lady vacuna,NEUTRO
104,"-""@19991007 @soylajefita Ojo",ojo,NEUTRO


## Lemmatizacion

In [6]:
import spacy
from nltk.stem.snowball import SnowballStemmer

sp = spacy.load('es_core_news_sm')

def lemmatization(text):
    doc = sp(text)
    return ' '.join([word.lemma_ for word in doc]) 

#stemmer = SnowballStemmer('spanish')
#stemmed_spanish = [stemmer.stem(item) for item in spanish_words]

In [7]:
df['lem_tweet'] = df['norm_tweet'].apply(lemmatization)
df['lemtokenized_tweet'] = df['lem_tweet'].apply(tokenize)
df[['Label', 'norm_tweet','lem_tweet','tokenized_tweet','lemtokenized_tweet']][100:110]

,Label,norm_tweet,lem_tweet,tokenized_tweet,lemtokenized_tweet
100,NEGATIVO,esos hermanos negrete una bola de vividores,ese hermano negretir uno bola de vividor,"[esos, hermanos, negrete, una, bola, de, vividores]","[ese, hermano, negretir, uno, bola, de, vividor]"
101,NEUTRO,esta en crisis la crisis url,este en crisis el crisis url,"[esta, en, crisis, la, crisis, url]","[este, en, crisis, el, crisis, url]"
102,NEGATIVO,uff cara triste pero aliviada microbio microbio quedate en casa casa con jardin coahuila mexico cara llorando,uff cara triste pero aliviado microbio microbio quedate en casa casa con jardin coahuila mexico caro llorar,"[uff, cara, triste, pero, aliviada, microbio, microbio, quedate, en, casa, casa, con, jardin, coahuila, mexico, cara, llorando]","[uff, cara, triste, pero, aliviado, microbio, microbio, quedate, en, casa, casa, con, jardin, coahuila, mexico, caro, llorar]"
103,NEUTRO,no se que tanto maman con lady vacuna,no él que tanto mamar con lady vacuna,"[no, se, que, tanto, maman, con, lady, vacuna]","[no, él, que, tanto, mamar, con, lady, vacuna]"
104,NEUTRO,ojo,ojo,[ojo],[ojo]
105,NEUTRO,hoy en el programa versiones raras,hoy en el programa versión rara,"[hoy, en, el, programa, versiones, raras]","[hoy, en, el, programa, versión, rara]"
106,NEUTRO,hoy participamos nuevamente para el programa de opinion yo creo,hoy participamos nuevamente para el programa de opinion yo creer,"[hoy, participamos, nuevamente, para, el, programa, de, opinion, yo, creo]","[hoy, participamos, nuevamente, para, el, programa, de, opinion, yo, creer]"
107,NEUTRO,flexibilidad,flexibilidad,[flexibilidad],[flexibilidad]
108,POSITIVO,dios los cuide y les conceda una pronta recuperacion y que cuide y bendiga a aquellos que aun no han presentado sintomas de contagio,dio él cuide y él concedar uno pronto recuperacion y que cuidir y bendiga a aquel que aun no haber presentar sintoma de contagio,"[dios, los, cuide, les, conceda, una, pronta, recuperacion, que, cuide, bendiga, aquellos, que, aun, no, han, presentado, sintomas, de, contagio]","[dio, él, cuide, él, concedar, uno, pronto, recuperacion, que, cuidir, bendiga, aquel, que, aun, no, haber, presentar, sintoma, de, contagio]"
109,NEGATIVO,y las sanciones para y por el terrible manejo de la crisis que,y el sanción para y por el terrible manejo de el crisis que,"[las, sanciones, para, por, el, terrible, manejo, de, la, crisis, que]","[el, sanción, para, por, el, terrible, manejo, de, el, crisis, que]"


## Stopwords

In [8]:
from nltk.corpus import stopwords

# Obtención de listado de stopwords del español
stop_words_esp = list(stopwords.words('spanish'))

def remove_stopwords(text):
    text = [w for w in text if not w in stop_words_esp]
    return text

df['normsw_tweet'] = df['norm_tweet'].apply(remove_stopwords)
df['lemsw_tweet'] = df['lem_tweet'].apply(remove_stopwords)
df['normtoksw_tweet'] = df['tokenized_tweet'].apply(remove_stopwords)
df['lemtoksw_tweet'] = df['lemtokenized_tweet'].apply(remove_stopwords)

print(stop_words_esp[:100])
df[['Label', 'lemtokenized_tweet', 'lemtoksw_tweet']][100:110]


['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo']


,Label,lemtokenized_tweet,lemtoksw_tweet
100,NEGATIVO,"[ese, hermano, negretir, uno, bola, de, vividor]","[hermano, negretir, bola, vividor]"
101,NEUTRO,"[este, en, crisis, el, crisis, url]","[crisis, crisis, url]"
102,NEGATIVO,"[uff, cara, triste, pero, aliviado, microbio, microbio, quedate, en, casa, casa, con, jardin, coahuila, mexico, caro, llorar]","[uff, cara, triste, aliviado, microbio, microbio, quedate, casa, casa, jardin, coahuila, mexico, caro, llorar]"
103,NEUTRO,"[no, él, que, tanto, mamar, con, lady, vacuna]","[mamar, lady, vacuna]"
104,NEUTRO,[ojo],[ojo]
105,NEUTRO,"[hoy, en, el, programa, versión, rara]","[hoy, programa, versión, rara]"
106,NEUTRO,"[hoy, participamos, nuevamente, para, el, programa, de, opinion, yo, creer]","[hoy, participamos, nuevamente, programa, opinion, creer]"
107,NEUTRO,[flexibilidad],[flexibilidad]
108,POSITIVO,"[dio, él, cuide, él, concedar, uno, pronto, recuperacion, que, cuidir, bendiga, aquel, que, aun, no, haber, presentar, sintoma, de, contagio]","[dio, cuide, concedar, pronto, recuperacion, cuidir, bendiga, aquel, aun, haber, presentar, sintoma, contagio]"
109,NEGATIVO,"[el, sanción, para, por, el, terrible, manejo, de, el, crisis, que]","[sanción, terrible, manejo, crisis]"


## Train test 

In [10]:
from sklearn.model_selection import train_test_split

X1 = df['norm_tweet']          #Tweets normalizados
X2 = df['lem_tweet']           #Tweets lemmatizados
X3 = df['tokenized_tweet']     #Normalizados y tokenizados
X4 = df['lemtokenized_tweet']  #Lemmatizados y tokenizados 
X5 = df['normtoksw_tweet']     #Normalizados, tokenizados y sin stopwords
X6 = df['lemtoksw_tweet']      #Lemmatizados, tokenizados y sin stopwords 
y = df['Label']                #Etiquetas

X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.25 ,random_state=37)

In [31]:
value, counts = np.unique(y_train, return_counts=True)
print(dict(zip(value, 100 * counts / sum(counts))))
value, counts = np.unique(y_test, return_counts=True)
print(dict(zip(value, 100 * counts / sum(counts))))

{'NEGATIVO': 33.64296081277213, 'NEUTRO': 44.93468795355588, 'POSITIVO': 21.42235123367199}
{'NEGATIVO': 34.89991296779809, 'NEUTRO': 44.38642297650131, 'POSITIVO': 20.713664055700608}


## Vectorizaciones

### Tf-idf

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf= TfidfVectorizer(min_df=3, ngram_range=(1,2), stop_words = stop_words_esp).fit(X_train)
                        
print('Numero de features: ' +str(len(tfidf.get_feature_names_out())))
tfidf.fit(X_train)

Numero de features: 2807


TfidfVectorizer(min_df=3, ngram_range=(1, 2),
                stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...])

In [34]:
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf  = tfidf.transform(X_test)
X_train_tfidf 

<3445x2807 sparse matrix of type '<class 'numpy.float64'>'
	with 30268 stored elements in Compressed Sparse Row format>

### Countvectorizer

In [36]:
from sklearn.feature_extraction.text import CountVectorizer

countvect = CountVectorizer(min_df=3, stop_words = stop_words_esp, ngram_range=(1,2)).fit(X_train)

print('Numero de features: ' +str(len(countvect.get_feature_names_out())))
countvect.fit(X_train)

Numero de features: 2807


CountVectorizer(min_df=3, ngram_range=(1, 2),
                stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...])

In [37]:
X_train_cv = countvect.transform(X_train)
X_test_cv  = countvect.transform(X_test)
X_train_cv

<3445x2807 sparse matrix of type '<class 'numpy.int64'>'
	with 30268 stored elements in Compressed Sparse Row format>

### Word embedding

In [38]:
from gensim.models import Word2Vec, KeyedVectors
from gensim.models import FastText

wordvectors_file = 'data/wiki.es.vec'
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file, limit=600000)

#embedding=200
#w2v = Word2Vec(X6, min_count=3, vector_size=embedding, window=5, sg=1 )
#w2v.train(X6, total_examples= len(df['lem_tweet']), epochs=20)

#wordvectors.most_similar('pozole')

In [39]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += wordvectors[word].reshape((1, size))
            count += 1.
        except KeyError:  
            continue
    if count != 0:
        vec /= count
    return vec

wordvec_arrays = np.zeros((len(X6), 300)) 
for i in range(len(X6)):
    wordvec_arrays[i,:] = word_vector(X6[i], 300)
    
X_w2v = pd.DataFrame(wordvec_arrays)
X_w2v.shape

(4594, 300)

In [40]:
# vocabulario CountVectorizer:
print(f" Número de tokens creados: {len(countvect.get_feature_names_out())}")
countvect.get_feature_names_out()

# vocabuilario TFIDF:
#print(f" Número de tokens creados: {len(tfidf.get_feature_names_out())}")
#tfidf.get_feature_names()

 Número de tokens creados: 2807


array(['abajo', 'abajo flechaber', 'abajo tono', ..., 'zeneca', 'zona',
       'zoom'], dtype=object)

## BERT

In [22]:
from bert_sklearn import BertClassifier
from bert_sklearn import load_model

model = BertClassifier()         # text/text pair classification

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)
scr = model.score(X_test, y_test)


Building sklearn text classifier...


100%|██████████| 231508/231508 [00:00<00:00, 13787537.88B/s]


Loading bert-base-uncased model...


100%|██████████| 433/433 [00:00<00:00, 147785.31B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 3101, validation data size: 344


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Training  :   0%|          | 0/97 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/bert_sklearn/model/pytorch_pretrained/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/python